In [1]:
from utils import log_to_token_seq, split_log
import pandas as pd


In [2]:
template_paths = [
    r"nulog\logs\Linux\Linux_2k.log_templates.csv",
    r"nulog\logs\Andriod\Andriod_2k.log_templates.csv",     
    r"nulog\logs\Apache\Apache_2k.log_templates.csv",
    r"nulog\logs\BGL\BGL_2k.log_templates.csv",
    r"nulog\logs\Hadoop\Hadoop_2k.log_templates.csv",
    r"nulog\logs\HDFS\HDFS_2k.log_templates.csv",
    r"nulog\logs\HealthApp\HealthApp_2k.log_templates.csv",
    r"nulog\logs\HPC\HPC_2k.log_templates.csv",
    r"nulog\logs\Mac\Mac_2k.log_templates.csv",
    r"nulog\logs\OpenSSH\OpenSSH_2k.log_templates.csv",
    r"nulog\logs\OpenStack\OpenStack_2k.log_templates.csv",
    r"nulog\logs\Proxifier\Proxifier_2k.log_templates.csv",
    r"nulog\logs\Spark\Spark_2k.log_templates.csv",
    r"nulog\logs\Thunderbird\Thunderbird_2k.log_templates.csv",
    r"nulog\logs\Windows\Windows_2k.log_templates.csv",
    r"nulog\logs\Zookeeper\Zookeeper_2k.log_templates.csv"
    ]

In [3]:
log_paths = [
    r"nulog\logs\Andriod\Andriod_2k.log",
    r"nulog\logs\Apache\Apache_2k.log",
    r"nulog\logs\BGL\BGL_2k.log",
    r"nulog\logs\Hadoop\Hadoop_2k.log",
    r"nulog\logs\HDFS\HDFS_2k.log",
    r"nulog\logs\HealthApp\HealthApp_2k.log",
    r"nulog\logs\HPC\HPC_2k.log",
    r"nulog\logs\Linux\Linux_2k.log",
    r"nulog\logs\Mac\Mac_2k.log",
    r"nulog\logs\OpenSSH\OpenSSH_2k.log",
    r"nulog\logs\OpenStack\OpenStack_2k.log",
    r"nulog\logs\Proxifier\Proxifier_2k.log",
    r"nulog\logs\Spark\Spark_2k.log",
    r"nulog\logs\Thunderbird\Thunderbird_2k.log",
    r"nulog\logs\Windows\Windows_2k.log",
    r"nulog\logs\Zookeeper\Zookeeper_2k.log"
]

In [4]:
dictionary = set()

for path in template_paths:

    df = pd.read_csv(path)

    for log in df["EventTemplate"]:
        tokens = split_log(log)
        
        for token in tokens:
            if token != '<*>':
                dictionary.add(token)

print(len(dictionary))

3193


In [5]:
data = []

In [6]:
max_length = 0
for log_path in log_paths:
    with open(log_path) as f:
        log = f.readline()
        while log:
            tok_seq = log_to_token_seq(log, 10)
            if len(tok_seq) > max_length:
                max_length = len(tok_seq)
            data.append(tok_seq)
            log = f.readline()

In [7]:
len(data)

32000

In [8]:
max_length

300

In [1]:
import torch
from torch.utils.data import Dataset
import random

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
torch.zeros(1).cuda()


AssertionError: Torch not compiled with CUDA enabled

In [11]:
class LogDataset(Dataset):
    def __init__(self, log_paths, vocab_size, max_length=300, mask_prob=0.15):
        self.log_paths = log_paths
        self.vocab_size = vocab_size
        self.max_length = max_length
        self.mask_prob = mask_prob
        self.special_offset = 10
    
        self.logs = []
        
        for log_path in self.log_paths:
            with open(log_path) as f:
                log = f.readline()
                while log:
                    tok_seq = log_to_token_seq(log, self.special_offset)
                    self.logs.append(tok_seq)
                    log = f.readline()

    def __len__(self):
        return len(self.logs)
    
    def __getitem__(self, idx):
        tokens = self.logs[idx][:self.max_length]
        input_ids = tokens + [0] * (self.max_length - len(tokens))  # Pad
        labels = [-100] * self.max_length  # Ignore positions not masked

        for i in range(len(tokens)):
            if random.random() < self.mask_prob:
                prob = random.random()
                labels[i] = input_ids[i]  # Store the original value

                if prob < 0.8:
                    input_ids[i] = 1  # [MASK]
                elif prob < 0.9:
                    input_ids[i] = random.randint(self.special_offset, self.vocab_size - 1)
                else:
                    pass  # Keep original

        attention_mask = [1 if t != 0 else 0 for t in input_ids]

        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.long)
        }

In [12]:
dataset = LogDataset(log_paths, 30000)
from torch.utils.data import DataLoader 
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

batch = next(iter(dataloader))
print(batch)

{'input_ids': tensor([[23053, 23834, 26216,  ...,     0,     0,     0],
        [23053, 23834, 26491,  ...,     0,     0,     0],
        [ 2266, 17428, 24948,  ...,     0,     0,     0],
        ...,
        [ 1216, 23834,  1145,  ...,     0,     0,     0],
        [24389,     1, 26030,  ...,     0,     0,     0],
        [ 2266, 16569,     1,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[ -100,  -100,  -100,  ...,  -100,  -100,  -100],
        [ -100,  -100,  -100,  ...,  -100,  -100,  -100],
        [ -100,  -100,  -100,  ...,  -100,  -100,  -100],
        ...,
        [ -100,  -100,  -100,  ...,  -100,  -100,  -100],
        [ -100, 22565,  -100,  ...,  -100,  -100,  -100],
        [ -100,  -100, 24948,  ...,  -100,  -100,  -100]])}


In [ ]:
from transformers import BertConfig, BertForMaskedLM, Trainer, TrainingArguments
from torch.utils.data import random_split

# Step 1: Define model config
config = BertConfig(
    vocab_size=30000,           # keep this as is
    max_position_embeddings=300,  # fine
    num_attention_heads=4,      # ↓ reduce attention heads
    num_hidden_layers=4,        # ↓ fewer transformer blocks
    type_vocab_size=1,
    hidden_size=256,            # ↓ smaller embedding and hidden dims
    intermediate_size=1024,     # ↓ smaller feed-forward layer
)
# Step 2: Create model
model = BertForMaskedLM(config)
model = model.to(device)

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


In [14]:
# Assuming `dataset` is your preprocessed LogTokenDataset
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=200,                 # add this if missing, evaluate less frequently
    learning_rate=5e-5,             # ↑ higher LR to compensate for smaller model
    per_device_train_batch_size=32, # ↑ larger batch size (if GPU allows)
    per_device_eval_batch_size=32,
    num_train_epochs=5,             # ↑ more epochs with smaller model is okay
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,               # ↓ log less frequently if needed
    save_steps=1000,                # ↓ save less often if you don’t need many checkpoints
    load_best_model_at_end=True,
)

C:\Users\I753367\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


In [17]:
trainer.train()

  0%|          | 0/5400 [00:00<?, ?it/s]

{'loss': 10.1093, 'grad_norm': 6.3579912185668945, 'learning_rate': 1.9962962962962963e-05, 'epoch': 0.01}


  0%|          | 0/200 [00:00<?, ?it/s]

{'eval_loss': 9.719738960266113, 'eval_runtime': 209.1562, 'eval_samples_per_second': 15.3, 'eval_steps_per_second': 0.956, 'epoch': 0.01}
{'loss': 9.515, 'grad_norm': 6.052066326141357, 'learning_rate': 1.9925925925925928e-05, 'epoch': 0.01}


  0%|          | 0/200 [00:00<?, ?it/s]

...

{'eval_loss': 6.30800724029541, 'eval_runtime': 529.9201, 'eval_samples_per_second': 6.039, 'eval_steps_per_second': 0.377, 'epoch': 0.19}
{'loss': 6.4295, 'grad_norm': 4.756584644317627, 'learning_rate': 1.8703703703703707e-05, 'epoch': 0.19}


  0%|          | 0/200 [00:00<?, ?it/s]

{'eval_loss': 6.228446006774902, 'eval_runtime': 796.3108, 'eval_samples_per_second': 4.019, 'eval_steps_per_second': 0.251, 'epoch': 0.19}
{'loss': 6.1601, 'grad_norm': 4.699578762054443, 'learning_rate': 1.866666666666667e-05, 'epoch': 0.2}


  0%|          | 0/200 [00:00<?, ?it/s]

KeyboardInterrupt: 